### Api Call für Meteostat

## Setup

In [1]:

import os
import time
import requests
import pandas as pd
from datetime import date, datetime, timedelta
from dateutil.parser import isoparse


In [2]:
AIR_BASE = "https://air-quality-api.open-meteo.com/v1/air-quality"

def openmeteo_air_quality(
    lat: float,
    lon: float,
    start: date,
    end: date,
    timezone: str = "Europe/Vienna",
    hourly: list[str] | None = None,
):
    if hourly is None:
        hourly = [
            "pm10", "pm2_5",
            "nitrogen_dioxide", "ozone", "sulphur_dioxide", "carbon_monoxide", "ammonia", "carbon_dioxide", "methane", "dust",
            "us_aqi", "european_aqi",
        ]

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start.isoformat(),
        "end_date": end.isoformat(),
        "timezone": timezone,
        "hourly": ",".join(hourly),
    }

    r = requests.get(AIR_BASE, params=params, timeout=30)
    r.raise_for_status()
    return r.json()


In [3]:
def air_quality_to_df(payload: dict) -> pd.DataFrame:
    hourly = payload.get("hourly", {})
    times = pd.to_datetime(hourly.get("time", []))
    df = pd.DataFrame({k: v for k, v in hourly.items() if k != "time"})
    df.insert(0, "time", times)
    df = df.set_index("time").sort_index()
    return df


In [4]:
lat, lon = 48.2082, 16.3738  # Wien

payload = openmeteo_air_quality(
    lat, lon,
    start=date(2025, 12, 5),
    end=date(2025, 12, 12),
    timezone="Europe/Vienna",
)

df = air_quality_to_df(payload)
df.head()


,pm10,pm2_5,nitrogen_dioxide,ozone,sulphur_dioxide,carbon_monoxide,ammonia,carbon_dioxide,methane,dust,us_aqi,european_aqi
time,,,,,,,,,,,,
2025-12-05 00:00:00,19.3,18.0,21.0,28.0,2.9,264.0,0.4,471,1532.0,0.0,83,62
2025-12-05 01:00:00,19.3,17.8,19.3,24.0,1.9,256.0,0.3,470,1537.0,0.0,82,61
2025-12-05 02:00:00,19.3,16.5,17.0,26.0,1.6,240.0,0.3,468,1531.0,0.0,81,61
2025-12-05 03:00:00,16.2,14.0,14.0,29.0,1.4,229.0,0.2,465,1519.0,0.0,80,61
2025-12-05 04:00:00,14.3,13.1,11.7,34.0,1.5,217.0,0.2,465,1512.0,0.0,79,61


In [5]:
df.describe()
df.to_csv("Data/vienna_air_quality.csv")
print("Saved: vienna_air_quality.csv")


Saved: vienna_air_quality.csv
